# 文件操作及数据处理

在大模型Python基础课程中已经讲过了，文件的基础操作，但是文件还有很多类型和格式，操作文件的核心其实是操作数据，处理数据。
本篇我们主要讲文件操作进阶、不同的数据格式、简单数据清洗。

## 一、文件操作进阶

### （一）、文件路径管理
文件路径分为绝对路径和相对路径，文件路径中（以windows为例），形如“C:\Program Files\7-Zip”就是绝对路径，是从盘符或者根路径开头的；相对路径形如“./project/test.py”或者“test.py”，要么就直接在当前工作目录文件夹下，可以直接用文件名，要么就用“./”开头，“.”的意思是当前目录，“..”的意思是上一级目录，比如“../database.py”的意思就是当前目录上一级目录下的一个名为database.py的文件。

在终端窗口中，我们一般使用cd “目标目录”来切换当前的工作目录。
查看当前目录的代码：

In [ ]:
import os
 
# 获取当前工作目录
current_dir = os.getcwd()
print(f"当前目录: {current_dir}")
 
# 组合路径 (更安全的方式)
data_dir = os.path.join(current_dir, "data")
print(f"数据目录: {data_dir}")
 
# 检查路径是否存在
if not os.path.exists(data_dir):
os.makedirs(data_dir)  # 创建目录

### （二）、文件读写模式详解
基础模式：
r：读取（默认）
w：写入（覆盖）
a：追加
x：排他创建（文件存在则失败）
+：更新（可读写）
b：二进制模式
t：文本模式（默认）
使用及混合示例：

In [ ]:
# 安全读取（文件不存在会报错）
with open("data.txt", "r") as file:
    content = file.read()
# 追加新内容（不会覆盖）
with open("log.txt", "a") as file:
    file.write("\nNew log entry")
# 读写二进制图片
with open("image.jpg", "rb") as img_in:
    with open("copy.jpg", "wb") as img_out:
        img_out.write(img_in.read())

### （三）、使用with语句的精髓
自动处理资源的打开和关闭，异常安全：

In [ ]:
# 传统方式 (需要手动关闭)
file = open("example.txt", "w")
try:
    file.write("Hello")
finally:
    file.close()  # 必须记住关闭!
 
# 现代方式 (推荐)
with open("example.txt", "w") as file:
    file.write("Hello")  # 离开with块时自动关闭
（四）、高效处理大文件
逐行读取避免内存溢出
# 处理大型日志文件
with open("large_log.txt", "r") as log_file:
    for line_num, line in enumerate(log_file, 1):
        if "ERROR" in line:
            print(f"在行 {line_num} 发现错误: {line.strip()}")

### （五）、文件指针操作

seek()和 tell()控制读取位置

In [ ]:
with open("data.txt", "r+") as file:
    # 读取前10字符
    start = file.read(10)
    print(f"开头内容: {start}")
    
    # 获取当前位置
    pos = file.tell()
    print(f"当前位置: {pos}")
    
    # 跳转到文件末尾
    file.seek(0, 2)  # 0:偏移量 2:从末尾计算
    file.write("\n新添加的行")

## 二、不同的数据格式

（一）、csv
csv数据是一种用来储存表格数据的纯文本格式，支持文本、数据及日期等数据类型，csv文件以逗号分隔的方式存储数据，通常以.csv作为文件扩展名。

Python内置csv模块处理表格数据

In [ ]:
import csv
 
# 读取CSV为字典列表
with open("students.csv", "r") as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        print(f"{row['姓名']} 的分数是 {row['分数']}")
 
# 写入CSV文件
data = [
    {"姓名": "张三", "年龄": 20, "专业": "计算机"},
    {"姓名": "李四", "年龄": 22, "专业": "数学"}
]
 
with open("students.csv", "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ["姓名", "年龄", "专业"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()  # 写入标题行
    writer.writerows(data)  # 写入所有数据

写入后：
<img src="images/image4.png" style="margin-left: 0px" width="1600px">

### （二）、JSON(JavaScript对象表示法)

JSON是一种轻量级的数据交换格式，易于人类阅读和编写，同时也便于机器解析和生成。它采用完全独立于编程语言的文本格式，但也使用了类似于C语言家族的约定。JSON是数据表示的理想选择，尤其在网络传输和存储过程中。

In [ ]:
import json
 
# 将Python对象转为JSON字符串
data = {
    "name": "Python学习小组",
    "members": 15,
    "active": True,
    "courses": ["基础语法", "文件操作", "数据处理"]
}
 
json_string = json.dumps(data, indent=2, ensure_ascii=False)
print("JSON字符串:")
print(json_string)
 
# 将JSON字符串写回文件
with open("group.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=2, ensure_ascii=False)
 
# 从文件读取JSON
with open("group.json", "r", encoding="utf-8") as json_file:
    loaded_data = json.load(json_file)
    print("\n从文件加载的数据:")
    print(f"小组名称: {loaded_data['name']}")
    print(f"课程列表: {', '.join(loaded_data['courses'])}")


### （三）、其他数据格式（简要介绍）
诸如word,pdf,excel等文件或者其他文件格式，一般常用的文件格式都有对应的python包，可以自行上网搜索，当然可能有的包并不能满足所有需求，这种时候就需要理解文件的底层存储格式，自己编写工具类来处理数据，比如pdf去水印就可以通过获取pdf所有元素的位置并且根据水印的特征（比如倾斜，透明，字体大小等）去除，并转换成txt。


## 三、简单数据清洗
数据清洗并不是一个具体的技术，它取决于要清洗的元数据的类型，格式或者说环境，一般来说是个算法上的问题。
（一）、识别和处理确实的值


In [ ]:
students = [
    {"id": 1, "name": "张三", "score": 85},
    {"id": 2, "name": "李四", "score": None},  # 缺失分数
    {"id": 3, "name": None, "score": 92},    # 缺失姓名
    {"id": 4, "name": "王五", "score": 78}
]


形如这种数据，识别出来None后，可以删掉该条数据或者填上默认值，比如name默认值可以设置为未知，score可以设置为60或者-1（如果不会报错的话）。

（二）、处理重复记录（去重）


In [4]:
orders = [
    {"order_id": "A100", "product": "鼠标"},
    {"order_id": "A101", "product": "键盘"},
    {"order_id": "A100", "product": "鼠标"},  # 重复订单
    {"order_id": "A102", "product": "显示器"}
]

去重可以用python里的set()，这种集合有不会重复的特征，也可以先排序，再遍历去重

（三）、数据类型转换
这个就是遍历或者其他方式遇到了，手动进行处理，比如某条数据是string要改成int，python里各个类型准换：
使用形如int(x)可以把一个其他类型的数据转化为int形，比如int(11.11)转化出来就是11。注意：如果要转换字符串“二十二”变成22是不行的，不知道为什么请自行思考
其他类型的格式转换请自行搜索，一般有需要找一下就行，不需要特别记。

（四）、数据过滤
比如有100个数字，标记了哪些不合法，比如小于零的去除，那么就遍历时去掉不合法的或者将合法的添加到一个新的集合里。注意：最好不要在遍历过程中删除原有集合元素，可能会报错。

总结：
处理数据可能会遇到各种情况，请根据实际情况分析，选择解决办法

